In [1]:
from qiskit import *
from qiskit import IBMQ
from qiskit.tools.visualization import plot_histogram
%matplotlib inline

In [41]:
sat_repr = [['-N1C1', '-N1C2', '-N1C3'], ['N1C1', 'N1C2'], ['N1C2', 'N1C3'], ['N1C1', 'N1C3'], ['-N2C1', '-N2C2', '-N2C3'], ['N2C1', 'N2C2'], ['N2C2', 'N2C3'], ['N2C1', 'N2C3'], ['-N3C1', '-N3C2', '-N3C3'], ['N3C1', 'N3C2'], ['N3C2', 'N3C3'], ['N3C1', 'N3C3'], ['N1C1', 'N2C1'], ['N1C2', 'N2C2'], ['N1C3', 'N2C3'], ['N1C1', 'N3C1'], ['N1C2', 'N3C2'], ['N1C3', 'N3C3'], ['N2C1', 'N3C1'], ['N2C2', 'N3C2'], ['N2C3', 'N3C3']]

variables = ["N1C{}".format(i) for i in range(1,4)]
variables = variables + ["N2C{}".format(i) for i in range(1,4)]
variables = variables + ["N3C{}".format(i) for i in range(1,4)]

# Number of qubits required
num_qubits = len(variables)

# Number of and gates required for final ands
# num_final_and_ancilla_bits = len(sat_repr) - 1

# Number of and gates required for intermediate ands
num_inter_and_ancilla_bits = len(sat_repr)

In [42]:
variables_to_qr_map = dict(zip(variables,[i for i in range(len(variables))]))
variables_to_qr_map

{'N1C1': 0,
 'N1C2': 1,
 'N1C3': 2,
 'N2C1': 3,
 'N2C2': 4,
 'N2C3': 5,
 'N3C1': 6,
 'N3C2': 7,
 'N3C3': 8}

In [59]:
qr = QuantumRegister(num_qubits)
out = AncillaRegister(1)
inter_ar = AncillaRegister(num_inter_and_ancilla_bits)


In [60]:
def not_and_3_elem(qc,qr1,qr2,qr3,ar):
    qc.x(qr1)
    qc.x(qr2)
    qc.x(qr3)
    qc.mct([qr1,qr2,qr3],ar)
    qc.x(qr1)
    qc.x(qr2)
    qc.x(qr3)
    qc.x(ar)

def reverse_not_and_3_elem(qc,qr1,qr2,qr3,ar):
    qc.x(ar)
    qc.x(qr1)
    qc.x(qr2)
    qc.x(qr3)
    qc.mct([qr1,qr2,qr3],ar)
    qc.x(qr1)
    qc.x(qr2)
    qc.x(qr3)

def not_of_and_result(qc,qr1,qr2,ar):
    qc.mct([qr1,qr2],ar)
    qc.x(ar)

def reverse_not_of_and_result(qc,qr1,qr2,ar):
    qc.x(ar)
    qc.mct([qr1,qr2],ar)

def add_circuit_for_clause(clause,qc,qr,inter_ar):
    
    if len(clause) == 3:
        not_and_3_elem(qc,qr[variables_to_qr_map[clause[0][1:]]],
                        qr[variables_to_qr_map[clause[1][1:]]],
                        qr[variables_to_qr_map[clause[2][1:]]],
                        inter_ar)
    else:
        not_of_and_result(qc,qr[variables_to_qr_map[clause[0]]],
                            qr[variables_to_qr_map[clause[1]]],
                            inter_ar)

def clean_ancilla_for_clause(clause,qc,qr,inter_ar):
    
    if len(clause) == 3:
        reverse_not_and_3_elem(qc,qr[variables_to_qr_map[clause[0][1:]]],
                        qr[variables_to_qr_map[clause[1][1:]]],
                        qr[variables_to_qr_map[clause[2][1:]]],
                        inter_ar)
    else:
        reverse_not_of_and_result(qc,qr[variables_to_qr_map[clause[0]]],
                                qr[variables_to_qr_map[clause[1]]],
                                inter_ar)

In [61]:
quantumCircuit = QuantumCircuit(qr,out,inter_ar)

# Superposition
for i in range(len(variables)):
    quantumCircuit.h(qr[i])

# Intermediate results
for i in range(len(sat_repr)):
    add_circuit_for_clause(sat_repr[i],quantumCircuit,qr,inter_ar[i])

# Doing an and of all the results
quantumCircuit.mct(inter_ar,out)

# Cleaning up ancillae bits
for i in range(len(sat_repr)):
    clean_ancilla_for_clause(sat_repr[i],quantumCircuit,qr,inter_ar[i])

quantumCircuit.draw()

┌───┐┌───┐     ┌───┐                                                  »
 q12_0: ┤ H ├┤ X ├──■──┤ X ├────────────■─────────────────────────────────────»
        ├───┤├───┤  │  ├───┤            │                                     »
 q12_1: ┤ H ├┤ X ├──■──┤ X ├────────────■───────────────────■─────────────────»
        ├───┤├───┤  │  ├───┤            │                   │                 »
 q12_2: ┤ H ├┤ X ├──■──┤ X ├────────────┼───────────────────■─────────────────»
        ├───┤├───┤  │  └───┘┌───┐       │                   │                 »
 q12_3: ┤ H ├┤ X ├──┼────■──┤ X ├───────┼────■──────────────┼─────────────────»
        ├───┤├───┤  │    │  ├───┤       │    │              │                 »
 q12_4: ┤ H ├┤ X ├──┼────■──┤ X ├───────┼────■──────────────┼────■────────────»
        ├───┤├───┤  │    │  ├───┤       │    │              │    │            »
 q12_5: ┤ H ├┤ X ├──┼────■──┤ X ├───────┼────┼──────────────┼────■────────────»
        ├───┤├───┤  │    │  └───┘┌───┐  │    │              │    │            »
 q12_6: ┤ H ├┤ X ├──┼────┼────■──┤ X ├──┼────┼────■─────────┼────┼────────────»
        ├───┤├───┤  │    │    │  ├───┤  │    │    │         │    │            »
 q12_7: ┤ H ├┤ X ├──┼────┼────■──┤ X ├──┼────┼────■─────────┼────┼────■───────»
        ├───┤├───┤  │    │    │  ├───┤  │    │    │         │    │    │       »
 q12_8: ┤ H ├┤ X ├──┼────┼────■──┤ X ├──┼────┼────┼─────────┼────┼────■───────»
        └───┘└───┘  │    │    │  └───┘  │    │    │         │    │    │       »
   a13: ────────────┼────┼────┼─────────┼────┼────┼─────────┼────┼────┼───────»
                  ┌─┴─┐  │    │  ┌───┐  │    │    │         │    │    │       »
 a14_0: ──────────┤ X ├──┼────┼──┤ X ├──┼────┼────┼─────────┼────┼────┼───────»
                  └───┘  │    │  └───┘┌─┴─┐  │    │  ┌───┐  │    │    │       »
 a14_1: ─────────────────┼────┼───────┤ X ├──┼────┼──┤ X ├──┼────┼────┼───────»
                         │    │       └───┘  │    │  └───┘┌─┴─┐  │    │  ┌───┐»
 a14_2: ─────────────────┼────┼──────────────┼────┼───────┤ X ├──┼────┼──┤ X ├»
                         │    │              │    │       └───┘  │    │  └───┘»
 a14_3: ─────────────────┼────┼──────────────┼────┼──────────────┼────┼───────»
                       ┌─┴─┐  │  ┌───┐       │    │              │    │       »
 a14_4: ───────────────┤ X ├──┼──┤ X ├───────┼────┼──────────────┼────┼───────»
                       └───┘  │  └───┘     ┌─┴─┐  │  ┌───┐       │    │       »
 a14_5: ──────────────────────┼────────────┤ X ├──┼──┤ X ├───────┼────┼───────»
                              │            └───┘  │  └───┘     ┌─┴─┐  │  ┌───┐»
 a14_6: ──────────────────────┼───────────────────┼────────────┤ X ├──┼──┤ X ├»
                              │                   │            └───┘  │  └───┘»
 a14_7: ──────────────────────┼───────────────────┼───────────────────┼───────»
                            ┌─┴─┐┌───┐            │                   │       »
 a14_8: ────────────────────┤ X ├┤ X ├────────────┼───────────────────┼───────»
                            └───┘└───┘          ┌─┴─┐┌───┐            │       »
 a14_9: ────────────────────────────────────────┤ X ├┤ X ├────────────┼───────»
                                                └───┘└───┘          ┌─┴─┐┌───┐»
a14_10: ────────────────────────────────────────────────────────────┤ X ├┤ X ├»
                                                                    └───┘└───┘»
a14_11: ──────────────────────────────────────────────────────────────────────»
                                                                              »
a14_12: ──────────────────────────────────────────────────────────────────────»
                                                                              »
a14_13: ──────────────────────────────────────────────────────────────────────»
                                                                              »
a14_14: ──────────────────────────────────────────────────────────────────────»
        

# Studying

In [21]:
qr = QuantumRegister(4)
out = QuantumRegister(1)
c_ar = AncillaRegister(4)
a_ar = AncillaRegister(2)

In [22]:
qc = QuantumCircuit(qr,out,c_ar,a_ar)
qc.draw()

q3_0: 
      
q3_1: 
      
q3_2: 
      
q3_3: 
      
  q4: 
      
a2_0: 
      
a2_1: 
      
a2_2: 
      
a2_3: 
      
a3_0: 
      
a3_1:

In [23]:
for i in range(4):
    qc.h(qr[i])

qc.x(out)
qc.h(out)
qc.draw()

┌───┐     
q3_0: ┤ H ├─────
      ├───┤     
q3_1: ┤ H ├─────
      ├───┤     
q3_2: ┤ H ├─────
      ├───┤     
q3_3: ┤ H ├─────
      ├───┤┌───┐
  q4: ┤ X ├┤ H ├
      └───┘└───┘
a2_0: ──────────
                
a2_1: ──────────
                
a2_2: ──────────
                
a2_3: ──────────
                
a3_0: ──────────
                
a3_1: ──────────

In [24]:
def copy_to_ancilla(qc,qr,ar):
    qc.x(qr)
    qc.x(ar)
    qc.cx(qr,ar)
    qc.x(qr)

In [25]:
copy_to_ancilla(qc,qr[1],c_ar[0])
qc.draw()

┌───┐               
q3_0: ┤ H ├───────────────
      ├───┤┌───┐     ┌───┐
q3_1: ┤ H ├┤ X ├──■──┤ X ├
      ├───┤└───┘  │  └───┘
q3_2: ┤ H ├───────┼───────
      ├───┤       │       
q3_3: ┤ H ├───────┼───────
      ├───┤┌───┐  │       
  q4: ┤ X ├┤ H ├──┼───────
      ├───┤└───┘┌─┴─┐     
a2_0: ┤ X ├─────┤ X ├─────
      └───┘     └───┘     
a2_1: ────────────────────
                          
a2_2: ────────────────────
                          
a2_3: ────────────────────
                          
a3_0: ────────────────────
                          
a3_1: ────────────────────

In [26]:
copy_to_ancilla(qc,qr[2],c_ar[1])
copy_to_ancilla(qc,qr[3],c_ar[2])
qc.draw()

┌───┐                         
q3_0: ┤ H ├─────────────────────────
      ├───┤┌───┐     ┌───┐          
q3_1: ┤ H ├┤ X ├──■──┤ X ├──────────
      ├───┤├───┤  │  └───┘┌───┐     
q3_2: ┤ H ├┤ X ├──┼────■──┤ X ├─────
      ├───┤├───┤  │    │  └───┘┌───┐
q3_3: ┤ H ├┤ X ├──┼────┼────■──┤ X ├
      ├───┤├───┤  │    │    │  └───┘
  q4: ┤ X ├┤ H ├──┼────┼────┼───────
      ├───┤└───┘┌─┴─┐  │    │       
a2_0: ┤ X ├─────┤ X ├──┼────┼───────
      ├───┤     └───┘┌─┴─┐  │       
a2_1: ┤ X ├──────────┤ X ├──┼───────
      ├───┤          └───┘┌─┴─┐     
a2_2: ┤ X ├───────────────┤ X ├─────
      └───┘               └───┘     
a2_3: ──────────────────────────────
                                    
a3_0: ──────────────────────────────
                                    
a3_1: ──────────────────────────────

In [27]:
qc.x(c_ar[3])
qc.mct([qr[0],qr[1]],c_ar[3])
qc.draw()

┌───┐                              
q3_0: ┤ H ├───────────────────────────■──
      ├───┤┌───┐     ┌───┐            │  
q3_1: ┤ H ├┤ X ├──■──┤ X ├────────────■──
      ├───┤├───┤  │  └───┘┌───┐       │  
q3_2: ┤ H ├┤ X ├──┼────■──┤ X ├───────┼──
      ├───┤├───┤  │    │  └───┘┌───┐  │  
q3_3: ┤ H ├┤ X ├──┼────┼────■──┤ X ├──┼──
      ├───┤├───┤  │    │    │  └───┘  │  
  q4: ┤ X ├┤ H ├──┼────┼────┼─────────┼──
      ├───┤└───┘┌─┴─┐  │    │         │  
a2_0: ┤ X ├─────┤ X ├──┼────┼─────────┼──
      ├───┤     └───┘┌─┴─┐  │         │  
a2_1: ┤ X ├──────────┤ X ├──┼─────────┼──
      ├───┤          └───┘┌─┴─┐       │  
a2_2: ┤ X ├───────────────┤ X ├───────┼──
      ├───┤               └───┘     ┌─┴─┐
a2_3: ┤ X ├─────────────────────────┤ X ├
      └───┘                         └───┘
a3_0: ───────────────────────────────────
                                         
a3_1: ───────────────────────────────────

In [28]:
qc.mct([c_ar[0],c_ar[1]],a_ar[0])
qc.mct([c_ar[2],a_ar[0]],a_ar[1])
qc.draw()

┌───┐                                   
q3_0: ┤ H ├───────────────────────────■───────
      ├───┤┌───┐     ┌───┐            │       
q3_1: ┤ H ├┤ X ├──■──┤ X ├────────────■───────
      ├───┤├───┤  │  └───┘┌───┐       │       
q3_2: ┤ H ├┤ X ├──┼────■──┤ X ├───────┼───────
      ├───┤├───┤  │    │  └───┘┌───┐  │       
q3_3: ┤ H ├┤ X ├──┼────┼────■──┤ X ├──┼───────
      ├───┤├───┤  │    │    │  └───┘  │       
  q4: ┤ X ├┤ H ├──┼────┼────┼─────────┼───────
      ├───┤└───┘┌─┴─┐  │    │         │       
a2_0: ┤ X ├─────┤ X ├──┼────┼────■────┼───────
      ├───┤     └───┘┌─┴─┐  │    │    │       
a2_1: ┤ X ├──────────┤ X ├──┼────■────┼───────
      ├───┤          └───┘┌─┴─┐  │    │       
a2_2: ┤ X ├───────────────┤ X ├──┼────┼────■──
      ├───┤               └───┘  │  ┌─┴─┐  │  
a2_3: ┤ X ├──────────────────────┼──┤ X ├──┼──
      └───┘                    ┌─┴─┐└───┘  │  
a3_0: ─────────────────────────┤ X ├───────■──
                               └───┘     ┌─┴─┐
a3_1: ───────────────────────────────────┤ X ├
                                         └───┘

In [29]:
qc.mct([a_ar[1],c_ar[3]],out)
qc.draw()

┌───┐                                        
q3_0: ┤ H ├───────────────────────────■────────────
      ├───┤┌───┐     ┌───┐            │            
q3_1: ┤ H ├┤ X ├──■──┤ X ├────────────■────────────
      ├───┤├───┤  │  └───┘┌───┐       │            
q3_2: ┤ H ├┤ X ├──┼────■──┤ X ├───────┼────────────
      ├───┤├───┤  │    │  └───┘┌───┐  │            
q3_3: ┤ H ├┤ X ├──┼────┼────■──┤ X ├──┼────────────
      ├───┤├───┤  │    │    │  └───┘  │       ┌───┐
  q4: ┤ X ├┤ H ├──┼────┼────┼─────────┼───────┤ X ├
      ├───┤└───┘┌─┴─┐  │    │         │       └─┬─┘
a2_0: ┤ X ├─────┤ X ├──┼────┼────■────┼─────────┼──
      ├───┤     └───┘┌─┴─┐  │    │    │         │  
a2_1: ┤ X ├──────────┤ X ├──┼────■────┼─────────┼──
      ├───┤          └───┘┌─┴─┐  │    │         │  
a2_2: ┤ X ├───────────────┤ X ├──┼────┼────■────┼──
      ├───┤               └───┘  │  ┌─┴─┐  │    │  
a2_3: ┤ X ├──────────────────────┼──┤ X ├──┼────■──
      └───┘                    ┌─┴─┐└───┘  │    │  
a3_0: ─────────────────────────┤ X ├───────■────┼──
                               └───┘     ┌─┴─┐  │  
a3_1: ───────────────────────────────────┤ X ├──■──
                                         └───┘